# From spreadsheet to Kobo — Part ✌️
---

In the following walk-through, we'll take a look at the more complex case of submitting data with repeat groups back to the Kobo server. The process will practically be the same for those without repeat groups, but some of the methods might require minor modifications.

We'll do some simple data cleaning from an Excel export and then transform it to a structure that will be easier to update the submissions' XML trees.

We will look at two approaches of submitting data to the server:
1. [Approach 1](#Approach-1:-Send-modified-data-to-a-cloned-form) will look at how to clone, deploy and submit to the cloned asset
2. [Approach 2](#Approach-2:-Send-modified-data-back-to-existing-form) will look at submitting data back to the original asset


## Get things setup 👌

![](https://media.giphy.com/media/l0MYQu0EH1TtRw8ko/giphy.gif)

### Import whatever you need 👇

In [1]:
import copy
import io
import os
import requests
import uuid
from datetime import datetime
from random import sample
from time import sleep
from typing import Tuple
from xml.etree import ElementTree as ET

import pandas as pd
import pytz

### Set up some helpful constants that we'll use below 🤘

Note that if you are taking the Docker approach to running your notebook, you can do the following to include environment variables:

```bash
docker run -p 8888:8888 -v ~/notebooks:/home/jovyan -e KOBO_TOKEN='<TOKEN>' jupyter/datascience-notebook
```

In [2]:
TOKEN = os.environ.get('KOBO_TOKEN')

In [3]:
KC_URL = 'https://kc.kobotoolbox.org'
KF_URL = 'https://kf.kobotoolbox.org'

ASSET_UID = 'abkTtAKoX7CK2pa6ToHyBV'

ASSETS_URL = f'{KF_URL}/api/v2/assets/'
DATA_URL = f'{ASSETS_URL}{ASSET_UID}/data'
DEPLOYMENT_URL = '{}{}/deployment/' #url template used below
XML_URL = f'{DATA_URL}.xml'
SUMISSION_URL = f'{KC_URL}/api/v1/submissions'
FORMS_URL = f'{KC_URL}/api/v1/forms'

HEADERS = {
    'Authorization': f'Token {TOKEN}'
}
PARAMS = {
    'format': 'json'
}

FILENAME = 'repeat_groups_example.xlsx'

ASSET_NAME = 'Repeat Groups Example'
CLONED_ASSET_NAME = f'Clone of {ASSET_NAME}'
REPEAT_GROUP_NAME = 'group_gp3qf47'

def _get_deployment_url(asset_uid: str) -> str:
    return DEPLOYMENT_URL.format(ASSETS_URL, asset_uid)

# Data! 📊
---

## Let's take a peek into our data 👀

![](https://media.giphy.com/media/Zvgb12U8GNjvq/giphy.gif)

In [4]:
subs = pd.read_excel(FILENAME)
subs.head()

,start,end,What_is_your_name,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags,_index
0,2021-03-31T16:49:17.317-07:00,2021-03-31T16:49:44.237-07:00,Octavian,90848911,5377d1f8-f176-4a0f-8bd9-06bcc095c77a,2021-03-31T23:49:48,NaN,[],submitted_via_web,NaN,NaN,1
1,2021-03-31T16:49:44.254-07:00,2021-03-31T16:50:08.340-07:00,Mark Antony,90848933,c10b537e-3caa-48be-825b-a94752564ab2,2021-03-31T23:50:19,NaN,[],submitted_via_web,NaN,NaN,2
2,2021-03-31T16:50:08.355-07:00,2021-03-31T16:50:31.644-07:00,Lepidus,90848939,9ce27ce0-f5c5-4924-ab00-b0874b7f05c8,2021-03-31T23:50:35,NaN,[],submitted_via_web,NaN,NaN,3


In [5]:
subs.columns.to_list()

['start',
 'end',
 'What_is_your_name',
 '_id',
 '_uuid',
 '_submission_time',
 '_validation_status',
 '_notes',
 '_status',
 '_submitted_by',
 '_tags',
 '_index']

In [6]:
subs_cols = ['What_is_your_name', '_uuid']
subs_filtered = subs[subs_cols]
subs_filtered.head()

,What_is_your_name,_uuid
0,Octavian,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
1,Mark Antony,c10b537e-3caa-48be-825b-a94752564ab2
2,Lepidus,9ce27ce0-f5c5-4924-ab00-b0874b7f05c8


In [7]:
repeat_subs = pd.read_excel(FILENAME, sheet_name=REPEAT_GROUP_NAME)
repeat_subs.head()

,A_pizza_place_you_like,Your_favourite_toppings_there,Your_favourite_toppings_there/cheese,Your_favourite_toppings_there/pepperoni,Your_favourite_toppings_there/avo,_index,_parent_table_name,_parent_index,_submission__id,_submission__uuid,_submission__submission_time,_submission__validation_status,_submission__notes,_submission__status,_submission__submitted_by,_submission__tags
0,Place 1,cheese pepperoni avo,1,1,1,1,Repeat groups 4,1,90848911,5377d1f8-f176-4a0f-8bd9-06bcc095c77a,2021-03-31T23:49:48,NaN,[],submitted_via_web,NaN,[]
1,Place 2,cheese pepperoni,1,1,0,2,Repeat groups 4,1,90848911,5377d1f8-f176-4a0f-8bd9-06bcc095c77a,2021-03-31T23:49:48,NaN,[],submitted_via_web,NaN,[]
2,Place 3,cheese avo,1,0,1,3,Repeat groups 4,1,90848911,5377d1f8-f176-4a0f-8bd9-06bcc095c77a,2021-03-31T23:49:48,NaN,[],submitted_via_web,NaN,[]
3,Place 1,cheese pepperoni avo,1,1,1,4,Repeat groups 4,2,90848933,c10b537e-3caa-48be-825b-a94752564ab2,2021-03-31T23:50:19,NaN,[],submitted_via_web,NaN,[]
4,Place 3,pepperoni avo,0,1,1,5,Repeat groups 4,2,90848933,c10b537e-3caa-48be-825b-a94752564ab2,2021-03-31T23:50:19,NaN,[],submitted_via_web,NaN,[]


## Let's do some data cleaning 🧹🧹🧹
---

![](https://media.giphy.com/media/3oKIPCSX4UHmuS41TG/giphy.gif)

There's no right or wrong way to do this, but for the purposes of the tutorial, we'll just make some simple changes the submission data.

In [8]:
repeat_subs_cols = ['A_pizza_place_you_like', 'Your_favourite_toppings_there', '_submission__uuid']
repeat_subs_filtered = repeat_subs[repeat_subs_cols]
repeat_subs_filtered.head()

,A_pizza_place_you_like,Your_favourite_toppings_there,_submission__uuid
0,Place 1,cheese pepperoni avo,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
1,Place 2,cheese pepperoni,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
2,Place 3,cheese avo,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
3,Place 1,cheese pepperoni avo,c10b537e-3caa-48be-825b-a94752564ab2
4,Place 3,pepperoni avo,c10b537e-3caa-48be-825b-a94752564ab2


In [9]:
USER = 'Octavian'
user_uid = subs_filtered[subs_filtered['What_is_your_name'] == USER]['_uuid'].values[0]
user_uid

'5377d1f8-f176-4a0f-8bd9-06bcc095c77a'

In [10]:
repeat_subs_for_user = repeat_subs_filtered[repeat_subs_filtered['_submission__uuid'] == user_uid]
repeat_subs_for_user

,A_pizza_place_you_like,Your_favourite_toppings_there,_submission__uuid
0,Place 1,cheese pepperoni avo,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
1,Place 2,cheese pepperoni,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
2,Place 3,cheese avo,5377d1f8-f176-4a0f-8bd9-06bcc095c77a


In [11]:
def remove_choice(string_choices: str, choice: str) -> str:
    return ' '.join([c for c in string_choices.split() if c != choice])

In [12]:
def map_place_names(place: str) -> str:
    places = {
        'Place 1': 'Limoncello',
        'Place 2': 'The Backyard',
        'Place 3': 'Basilico'
    }
    new_place = places.get(place)
    return new_place if new_place is not None else place

In [13]:
all_uuids = subs_filtered['_uuid'].tolist()
uuids_for_updating = sample(all_uuids, 2)
uuids_for_updating

['9ce27ce0-f5c5-4924-ab00-b0874b7f05c8',
 'c10b537e-3caa-48be-825b-a94752564ab2']

In [14]:
for i, row in repeat_subs_filtered.iterrows():
    if row['_submission__uuid'] in uuids_for_updating:
        row['Your_favourite_toppings_there'] = remove_choice(row['Your_favourite_toppings_there'], 'pepperoni')
    row['A_pizza_place_you_like'] = map_place_names(row['A_pizza_place_you_like'])

In [15]:
repeat_subs_filtered

,A_pizza_place_you_like,Your_favourite_toppings_there,_submission__uuid
0,Limoncello,cheese pepperoni avo,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
1,The Backyard,cheese pepperoni,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
2,Basilico,cheese avo,5377d1f8-f176-4a0f-8bd9-06bcc095c77a
3,Limoncello,cheese avo,c10b537e-3caa-48be-825b-a94752564ab2
4,Basilico,avo,c10b537e-3caa-48be-825b-a94752564ab2
5,The Backyard,cheese avo,9ce27ce0-f5c5-4924-ab00-b0874b7f05c8
6,Basilico,cheese avo,9ce27ce0-f5c5-4924-ab00-b0874b7f05c8


## Massage the data 💆‍♀️

![](https://media.giphy.com/media/JgUNowqwNo5FK/giphy.gif)

Let's massage that cleaned data back into a format that's more useful to work with.

**Note that this will need to be adjusted to your particular needs**

In [16]:
updated_subs = []
for i, subs_row in subs_filtered.iterrows():
    sub = {
        'What_is_your_name': subs_row['What_is_your_name'],
        '_uuid': subs_row['_uuid']
    }
    
    df_repeats = repeat_subs_filtered[repeat_subs_filtered['_submission__uuid'] == sub['_uuid']]
    repeat_group = []
    for j, repeats_row in df_repeats.iterrows():
        repeat_group.append({
            'A_pizza_place_you_like': repeats_row['A_pizza_place_you_like'],
            'Your_favourite_toppings_there': repeats_row['Your_favourite_toppings_there']
        })
    sub[REPEAT_GROUP_NAME] = repeat_group
    updated_subs.append(sub)

In [17]:
updated_subs

[{'What_is_your_name': 'Octavian',
  '_uuid': '5377d1f8-f176-4a0f-8bd9-06bcc095c77a',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'cheese pepperoni avo'},
   {'A_pizza_place_you_like': 'The Backyard',
    'Your_favourite_toppings_there': 'cheese pepperoni'},
   {'A_pizza_place_you_like': 'Basilico',
    'Your_favourite_toppings_there': 'cheese avo'}]},
 {'What_is_your_name': 'Mark Antony',
  '_uuid': 'c10b537e-3caa-48be-825b-a94752564ab2',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'Limoncello',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'Basilico',
    'Your_favourite_toppings_there': 'avo'}]},
 {'What_is_your_name': 'Lepidus',
  '_uuid': '9ce27ce0-f5c5-4924-ab00-b0874b7f05c8',
  'group_gp3qf47': [{'A_pizza_place_you_like': 'The Backyard',
    'Your_favourite_toppings_there': 'cheese avo'},
   {'A_pizza_place_you_like': 'Basilico',
    'Your_favourite_toppings_there': 'cheese avo'}]}]

## Grab all the submissions 🛒

![](https://media.giphy.com/media/PjJ1XUXFkp6FRA2SrB/giphy.gif)

Just as in Part 1, we need to grab the XML to update it with our newly cleaned data.

In [18]:
res = requests.get(url=XML_URL, headers=HEADERS, params=PARAMS)

In [19]:
res.status_code

200

In [20]:
xml_str = res.text

In [21]:
parsed_xml = ET.fromstring(xml_str)

Let's take look at what a single submission's XML looks like

In [22]:
e = parsed_xml.find(f'results/{ASSET_UID}')
print(ET.tostring(e).decode())

<abkTtAKoX7CK2pa6ToHyBV id="abkTtAKoX7CK2pa6ToHyBV" version="1 (2021-03-31 23:49:07)">
          <formhub>
            <uuid>70b4c958db894e2d94620129fafdae51</uuid>
          </formhub>
          <start>2021-03-31T16:49:17.317-07:00</start>
          <end>2021-03-31T16:49:44.237-07:00</end>
          <What_is_your_name>Octavian</What_is_your_name>
          <group_gp3qf47>
            <A_pizza_place_you_like>Place 1</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese pepperoni avo</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Place 2</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese pepperoni</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Place 3</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese avo</Your_favourite_toppings_there>
          </group_gp3qf47>
          <__version__>vnGjnM3Hc

In [23]:
e.findall('group_gp3qf47')

[<Element 'group_gp3qf47' at 0x7fd5bce796d0>,
 <Element 'group_gp3qf47' at 0x7fd5bce798b0>,
 <Element 'group_gp3qf47' at 0x7fd5bce799f0>]

Let's grab all the submissions contained in the `results` array

In [24]:
all_xml_subs = parsed_xml.findall(f'results/{ASSET_UID}')
all_xml_subs

[<Element 'abkTtAKoX7CK2pa6ToHyBV' at 0x7fd5bce790e0>,
 <Element 'abkTtAKoX7CK2pa6ToHyBV' at 0x7fd5bce79cc0>,
 <Element 'abkTtAKoX7CK2pa6ToHyBV' at 0x7fd5bce2d3b0>]

A helpful method to filter the submissions based on `uuid` that we'll use later on

In [25]:
def get_xml_for_submission_uid(xml: ET.Element, submission_uid: str) -> ET.Element:
    return [x for x in xml if x.find('meta/instanceID').text == f'uuid:{submission_uid}'][0]

In [26]:
sub_uuid = updated_subs[0]['_uuid']
sub_uuid

'5377d1f8-f176-4a0f-8bd9-06bcc095c77a'

In [27]:
tmp_el = get_xml_for_submission_uid(all_xml_subs, sub_uuid)
tmp_el

<Element 'abkTtAKoX7CK2pa6ToHyBV' at 0x7fd5bce790e0>

In [28]:
tmp_el.attrib

{'id': 'abkTtAKoX7CK2pa6ToHyBV', 'version': '1 (2021-03-31 23:49:07)'}

In [29]:
print(ET.tostring(tmp_el).decode())

<abkTtAKoX7CK2pa6ToHyBV id="abkTtAKoX7CK2pa6ToHyBV" version="1 (2021-03-31 23:49:07)">
          <formhub>
            <uuid>70b4c958db894e2d94620129fafdae51</uuid>
          </formhub>
          <start>2021-03-31T16:49:17.317-07:00</start>
          <end>2021-03-31T16:49:44.237-07:00</end>
          <What_is_your_name>Octavian</What_is_your_name>
          <group_gp3qf47>
            <A_pizza_place_you_like>Place 1</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese pepperoni avo</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Place 2</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese pepperoni</Your_favourite_toppings_there>
          </group_gp3qf47><group_gp3qf47>
            <A_pizza_place_you_like>Place 3</A_pizza_place_you_like>
            <Your_favourite_toppings_there>cheese avo</Your_favourite_toppings_there>
          </group_gp3qf47>
          <__version__>vnGjnM3Hc

# Hack together some methods 🔪
---

Some of these are just copied over from Part 1, some are extended and the rest are some minor additions.

In [30]:
def submit_data(xml_sub: bytes, _uuid: str) -> str:
    """
    Send the XML to kobo!
    """
    file_tuple = (_uuid, io.BytesIO(xml_sub))
    files = {'xml_submission_file': file_tuple}
    res = requests.Request(
        method='POST', url=SUMISSION_URL, files=files, headers=HEADERS
    )
    session = requests.Session()
    res = session.send(res.prepare())
    return res.status_code

def format_openrosa_datetime() -> str:
    """
    This is required to get the correct datetime formatting
    """
    return datetime.now(tz=pytz.UTC).isoformat('T', 'milliseconds')

In [31]:
def update_element_value(e: ET.Element, name: str, value: str) -> None:
    """
    Get or create a node and give it a value, even if nested within a group
    """
    el = e.find(name)
    if el is None:
        # this only works for questions nested within a single group
        # and will need to be extended to handle deeply nested groups
        # refer here for how this is implemented in KPI:
        # https://github.com/kobotoolbox/kpi/blob/27cff694f9a63fd091ba8b803ac925f77b8f513b/kpi/deployment_backends/kobocat_backend.py#L827-L857
        if '/' in name:
            root, node = name.split('/')
            el = ET.SubElement(e.find(root), node)
        else:
            el = ET.SubElement(e, name)
    el.text = value
    
def update_list_of_element_values(e: ET.Element, name: str, items: str) -> None:
    """
    Update a list of nodes within a group -- will fail if the item doesn't exist
    on the XML tree so extend appropriately.
    
    This takes care of updating the repeat group questions.
    """
    els = e.findall(name)
    for el, item in zip(els, items):
        for k, v in item.items():
            el.find(k).text = v
            
def update_root_element_tag_and_attrib(e: ET.Element, tag: str, attrib: dict) -> None:
    """
    Update the root of each submission's XML tree when submitting to a cloned form
    """
    e.tag = tag
    e.attrib = attrib

In [32]:
def create_submissions(xml: ET.Element, subs: list, clone_data=None) -> list:
    """
    Take a bunch of submissions and send them off
    
    If you're wanting to use a cloned project, the dict must contain the following data:
    
    clone_data = {
        'clone_asset_uid': 'azNBzazvrjaqmmgF2H7gF5',
        'version': '1 (2021-03-29 19:40:28)',
        '__version__': 'vuoUnu6ZRrYwbxGz48QLSZ',
        'formhub_uuid': '30f53e5a4bc0425fb5bb27819c14ba44'
    }
    
    """
    
    all_subs = []
    for sub in subs:
        sub = copy.deepcopy(sub)
        sub_uuid = sub.pop('_uuid')
        parsed_xml = get_xml_for_submission_uid(xml, sub_uuid)
        
        _now = format_openrosa_datetime()
        _uuid = str(uuid.uuid4())
        
        for k, v in sub.items():
            if isinstance(v, list):
                update_list_of_element_values(parsed_xml, k, v)
            else:
                update_element_value(parsed_xml, k, v)
        
        # We have to create a meta/deprecatedID node with the value of the old instanceID and then 
        # update the instanceID for the submission to be updated correctly on the server -- ONLY IF 
        # NOT USING A CLONED FORM
        if clone_data is None:
            update_element_value(parsed_xml, 'meta/deprecatedID', parsed_xml.find('meta/instanceID').text)
        update_element_value(parsed_xml, 'meta/instanceID', f'uuid:{_uuid}')
        
        # Updating the `start` and `end` times is not really necessary, but 
        # probably something you'd want to do
        update_element_value(parsed_xml, 'start', _now)
        update_element_value(parsed_xml, 'end', _now)
        
        if clone_data is not None:
            new_attrib = {
                'id': clone_data['clone_asset_uid'],
                'version': clone_data['version']
            }
            update_root_element_tag_and_attrib(parsed_xml, clone_data['clone_asset_uid'], new_attrib)
            update_element_value(parsed_xml, '__version__', clone_data['__version__'])
            update_element_value(parsed_xml, 'formhub/uuid', clone_data['formhub_uuid'])
        
        all_subs.append(submit_data(ET.tostring(parsed_xml), _uuid))
        
        # If you are submitting a large amount of data, please be mindful that it can
        # overwhelm the servers if sent in a short span of time. Letting it sleep for
        # for a short stint between each upload will be much appreciated
        sleep(0.2)
        
    return all_subs

# Approach 1: Send modified data to a cloned form

To preserve the integrity of your original submission data, this would be the recommended approach. If you have made changes to the data and want it stored back on the Kobo servers, you want to be careful not to corrupt your survey data and rather have a clean slate to work from — without fear of data loss.

We'll follow these steps:
1. [Clone the form](#1.-Clone-form)
2. [Deploy the cloned form](#2.-Deploy-the-cloned-form)
3. [Get deployment data from KC and KPI](#3.-Get-deployment-data)
4. [Send it off](#4.-Let's-send-that-data-off!)

## 1. Clone your form 🐑 🐑

![](https://media.giphy.com/media/TlK63EA6F1qRb7lll6M/giphy.gif)

Let's clone the original form and give it new name. For this example, I've included a UUID with each name just to ensure that it's unique, but this isn't necessary.

In [33]:
data = {
    'clone_from': ASSET_UID,
    'name': f'{CLONED_ASSET_NAME} {uuid.uuid4()}'
}
res = requests.post(url=ASSETS_URL, headers=HEADERS, params=PARAMS, data=data)

In [34]:
res.status_code

201

In [35]:
cloned_survey = res.json()

In [36]:
cloned_asset_uid = cloned_survey['uid']
cloned_asset_uid

'aciDTwqQz6uf7FMUEjFE4v'

## 2. Deploy the cloned form 🚀

![](https://media.giphy.com/media/k57ESX08OH3syRZn0a/giphy.gif)

In [37]:
data = {
    'active': 'true'
}
res = requests.post(url=_get_deployment_url(cloned_asset_uid), headers=HEADERS, params=PARAMS, data=data)

In [38]:
res.status_code

200

## 3. Get the deployment data 📖

### 3.1 Let's start with getting data from KC

In [39]:
res = requests.get(url=FORMS_URL, headers=HEADERS, params=PARAMS)

In [40]:
res.status_code

200

In [41]:
all_forms = res.json()

In [42]:
latest_form = [f for f in all_forms if f['id_string'] == cloned_asset_uid][0]
latest_form

{'url': 'https://kc.kobotoolbox.org/api/v1/forms/647820?format=json',
 'formid': 647820,
 'metadata': [],
 'owner': 'joshuaberetta',
 'public': False,
 'public_data': False,
 'require_auth': False,
 'tags': [],
 'title': 'Clone of Repeat Groups Example 892ebbd1-b2f0-469b-882b-fc3f8eaf4d65',
 'users': [{'user': 'joshuaberetta',
   'permissions': ['add_datadictionary',
    'add_xform',
    'change_datadictionary',
    'change_xform',
    'delete_data_xform',
    'delete_datadictionary',
    'delete_xform',
    'move_xform',
    'report_xform',
    'transfer_xform',
    'validate_xform',
    'view_xform']}],
 'hash': 'md5:0da1648fef7696adbf29dda7afd31333',
 'has_kpi_hooks': False,
 'description': 'Clone of Repeat Groups Example 892ebbd1-b2f0-469b-882b-fc3f8eaf4d65',
 'downloadable': True,
 'allows_sms': False,
 'encrypted': False,
 'sms_id_string': 'aciDTwqQz6uf7FMUEjFE4v',
 'id_string': 'aciDTwqQz6uf7FMUEjFE4v',
 'date_created': '2021-03-31T23:53:02.638969Z',
 'date_modified': '2021-03-3

In [43]:
formhub_uuid = latest_form['uuid']
formhub_uuid

'2a99f72ac2eb409db1b0e030339f0da6'

### 3.2 Now from KPI

There are a few things we need to update on each submission's XML:
- The root `tag` and attributes of `id` and `version`
- the value of `formhub/uuid` with the new uuid
- value of `__version__`

In [44]:
res = requests.get(f'{ASSETS_URL}{cloned_asset_uid}', headers=HEADERS, params=PARAMS)

In [45]:
res.status_code

200

In [46]:
deployed_versions = res.json()['deployed_versions']
deployed_versions

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'uid': 'v8vByYsmaxTVbtF3S6hLHg',
   'url': 'https://kf.kobotoolbox.org/api/v2/assets/aciDTwqQz6uf7FMUEjFE4v/versions/v8vByYsmaxTVbtF3S6hLHg/?format=json',
   'content_hash': 'b47ac0203f253524aaada90e13575b5a0041ee7b',
   'date_deployed': '2021-03-31T23:53:00.510740Z',
   'date_modified': '2021-03-31 23:53:00.510740+00:00'}]}

We'll need some helper functions to get what we want out of the API data:

In [47]:
def format_date_string(date_str: str) -> str:
    """
    Format goal: "1 (2021-03-29 19:40:28)"
    """
    date, time = date_str.split('T')
    return f"{date} {time.split('.')[0]}"

In [48]:
def get_info_from_deployed_versions(deployed_versions: dict) -> Tuple[str, str]:
    """
    Get the version formats
    """
    count = deployed_versions['count']
    
    latest_deployment = deployed_versions['results'][0]
    date = latest_deployment['date_deployed']
    version = latest_deployment['uid']
    
    return version, f'{count} ({format_date_string(date)})'

In [49]:
get_info_from_deployed_versions(deployed_versions)

('v8vByYsmaxTVbtF3S6hLHg', '1 (2021-03-31 23:53:00)')

Now let's construct the data we're going to use to update the submissions' XML before sending off to the newly created clone of the original asset.

In [50]:
_v_, v = get_info_from_deployed_versions(deployed_versions)
clone_data = {
        'clone_asset_uid': cloned_asset_uid,
        'version': v,
        '__version__': _v_,
        'formhub_uuid': formhub_uuid
    }

# 4. Let's send that data off! 🌈

![](https://media.giphy.com/media/qJuTECs2j8iX3TqNGv/giphy.gif)

**Remember to include that `clone_data` dictionary we just created otherwise it will overwrite the existing data**

In [51]:
parsed_xml = ET.fromstring(xml_str)
all_xml_subs = parsed_xml.findall(f'results/{ASSET_UID}')

In [52]:
responses = create_submissions(all_xml_subs, updated_subs, clone_data=clone_data)

In [53]:
all(res == 201 for res in responses)

True

In [54]:
pd.Series(responses).value_counts()

201    3
dtype: int64

### Success!! 🎉 (hopefully)

![](https://media.giphy.com/media/mQG644PY8O7rG/giphy.gif)

# Approach 2: Send modified data back to existing form
---

![](https://media.giphy.com/media/8cSRVVIa7QkNZmo939/giphy.gif)

**Word of caution: this will overwrite your existing submissions, so be very sure that you know what you're doing otherwise you will corrupt your data.**

The only difference in usage from [Approach 1](#Approach-1:-Send-modified-data-to-cloned-form) is that `clone_data` is not included in the function call.

In [55]:
parsed_xml = ET.fromstring(xml_str)
all_xml_subs = parsed_xml.findall(f'results/{ASSET_UID}')

In [56]:
responses = create_submissions(all_xml_subs, updated_subs)

In [57]:
all(res == 201 for res in responses)

True

In [58]:
pd.Series(responses).value_counts()

201    3
dtype: int64

## All done 🔥

Check the data table in the Kobo UI and ensure that your data has been successfully submitted.

![](https://media.giphy.com/media/lD76yTC5zxZPG/giphy.gif)